In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("JDBC_CATALOG") \
    .master("local[*]") \
    .getOrCreate()

print("SparkSession ready!")

SparkSession ready!


### Attention for the first time, spark doesn't anything about new catalog!

In [2]:
spark.sql("show catalogs").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [3]:
spark.sql("SHOW TABLES IN mysql_catalog.habr").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|     habr|   orders|      false|
+---------+---------+-----------+



### But, after first call, you can see it.

In [4]:
spark.sql("show catalogs").show()

+-------------+
|      catalog|
+-------------+
|mysql_catalog|
|spark_catalog|
+-------------+



In [5]:
spark.sql("select * from mysql_catalog.habr.orders").show()

+---+------------+-----------+----------+--------+------+-------------------+----------+-------------------+-------------------+
| id|    order_id|customer_id|product_id|quantity| price|         order_date|    status|         created_at|         updated_at|
+---+------------+-----------+----------+--------+------+-------------------+----------+-------------------+-------------------+
|  1|ORD-2024-001|        101|         1|       2|150.00|2024-01-15 10:30:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
|  2|ORD-2024-002|        102|         2|       1| 89.99|2024-01-18 14:20:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
|  3|ORD-2024-003|        103|         1|       3|150.00|2024-01-22 09:15:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
|  4|ORD-2024-004|        104|         3|       1|299.99|2024-01-25 16:45:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
|  5|ORD-2024-005|        105|         2|       2| 89.99|2024-01-28 11:30:00|delivered |2025-10-2

In [6]:
orders = spark.read.table("mysql_catalog.habr.orders")
orders_filtered = orders.filter("product_id > 1")
orders_filtered.explain('formatted')

== Physical Plan ==
* Project (2)
+- * Scan org.apache.spark.sql.execution.datasources.v2.jdbc.JDBCScan$$anon$1@654ee536  (1)


(1) Scan org.apache.spark.sql.execution.datasources.v2.jdbc.JDBCScan$$anon$1@654ee536  [codegen id : 1]
Output [10]: [id#121, order_id#122, customer_id#123, product_id#124, quantity#125, price#126, order_date#127, status#128, created_at#129, updated_at#130]
PushedFilters: [product_id IS NOT NULL, product_id > 1]
ReadSchema: struct<id:int,order_id:string,customer_id:int,product_id:int,quantity:int,price:decimal(10,2),order_date:timestamp,status:string,created_at:timestamp,updated_at:timestamp>

(2) Project [codegen id : 1]
Output [10]: [id#121, order_id#122, customer_id#123, product_id#124, quantity#125, price#126, order_date#127, staticinvoke(class org.apache.spark.sql.catalyst.util.CharVarcharCodegenUtils, StringType, readSidePadding, status#128, 10, true, false, true) AS status#131, created_at#129, updated_at#130]
Input [10]: [id#121, order_id#122, customer_

In [7]:
from pyspark.sql.functions import col

orders.filter(col("id") >= 10).show()

+---+------------+-----------+----------+--------+------+-------------------+----------+-------------------+-------------------+
| id|    order_id|customer_id|product_id|quantity| price|         order_date|    status|         created_at|         updated_at|
+---+------------+-----------+----------+--------+------+-------------------+----------+-------------------+-------------------+
| 10|ORD-2024-010|        110|         6|       1|450.00|2024-02-20 17:25:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
| 11|ORD-2024-011|        111|         3|       2|299.99|2024-03-05 10:15:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
| 12|ORD-2024-012|        112|         7|       1|125.75|2024-03-10 14:40:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
| 13|ORD-2024-013|        113|         1|       4|150.00|2024-03-15 09:30:00|delivered |2025-10-20 20:05:20|2025-10-20 20:05:20|
| 14|ORD-2024-014|        114|         8|       1| 89.99|2024-03-18 16:20:00|delivered |2025-10-2